In [1]:
# Data Collection

!pip install kaggle --upgrade
!pip3 install Pillow

import kaggle
import os
import json
import random
from datetime import datetime
from PIL import Image, ImageStat
from PIL.ExifTags import TAGS
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

def get_predominant_colors(image_file, numcolors=3, resize=150, output = []):
    # Resize image to speed up processing
    img = Image.open(image_file)
    img = img.copy()
    img.thumbnail((resize, resize))
 
    # Reduce to palette
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)
 
    # Find dominant colors
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    colors = list()
    for i in range(numcolors):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        colors.append(tuple(dominant_color))
    return colors

def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []    
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)    
    distance, index = kdt_db.query(rgb_tuple)
    return int(index)

ModuleNotFoundError: No module named 'webcolors'

In [ ]:
## You have to collect and download a set of images. You have the following tasks to program, automating the process as much as possible:
### 1. Create a folder called images.

!mkdir -p ./images
!mkdir -p ./metadata

### 2. Download open-licensed images to the folder images (minimum 100 images).

kaggle.api.authenticate()
kaggle.api.dataset_download_files('nielspace/pexels-mountain-images', path='./images', unzip=True)    
    
### 3. Save metadata of every image like image size, image format (.jpeg, .png, etc.), image orientation (landscape, portrait, square, etc.), creation date, camera model, etc. in one or more JSON files. You can make use of the Exif information present in the image files.

directory = './images/Mountain'
metadata = {}
orientation = ""
images_data =[]
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        image_data = {}
        path, file_extension = os.path.splitext(f)
        time_creation = os.path.getmtime(f)
        im = Image.open(f)
        size = im.size
        colors = {}
        predominant_colors = get_predominant_colors(directory + "/" + filename)
        colors["predominant_colors"] = []
        for predominant_color in predominant_colors:
            colors["predominant_colors"].append(convert_rgb_to_names(predominant_color))
        for idx,val in enumerate(["red","green","blue"]):
            colors[val] = Image.Image.getextrema(im)[idx]
            colors[val] = colors[val] + (ImageStat.Stat(im).median[idx],)
        if size[0]>size[1] :
            orientation = "landscape"
        elif size[0] == size[1] :
            orientation = "square"
        else :
            orientation = "portrait"
        metadata[filename] = {
            "file_extension": file_extension,
            "creation_date": datetime.fromtimestamp(time_creation/1000.0).strftime("%m/%d/%Y, %H:%M:%S"),
            "size": {
                "width":size[0],
                "height":size[1]
            },
            "orientation": orientation,
            "colors": {
                "predominant_colors":{
                    "first_color":colors["predominant_colors"][0],
                    "second_color":colors["predominant_colors"][1],
                    "third_color":colors["predominant_colors"][2]
                },
                "red": {
                    "minimum":colors["red"][0],
                    "maximum":colors["red"][1],
                    "median":colors["red"][2]
                },
                "green": {
                    "minimum":colors["green"][0],
                    "maximum":colors["green"][1],
                    "median":colors["green"][2]
                },
                "blue": {
                    "minimum":colors["blue"][0],
                    "maximum":colors["blue"][1],
                    "median":colors["blue"][2]
                }
            }
        }
        print(metadata[filename])


with open("./metadata/metadata.json", "w") as outfile:
    json.dump(metadata, outfile, indent=4)

        

In [ ]:
# Labeling and Annotation

directory = './images/Mountain'
data_path = "./metadata/metadata.json"


if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)
        
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        tags = {}
        tags["like"] = random.randint(0,100)
        tags["hashtag"] = "moutain"
        json_data[filename]["tags"] = tags
        
with open("./metadata/metadata.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)

In [4]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Data Analyses

numb_users = 1
directory = './images/Mountain'
data_path = "./metadata/metadata.json"
rand = 0

# Metadata for each file
if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)

# Creating data for each user
json_data_users = {}        
for i in range (0, numb_users):
    images_per_users_rand = []
    images_per_users_orientation = []
    images_per_users_colors = []
    tags_per_users = []
    rand = random.randint(1, 4)
    if rand == 1:
        tags_per_users.append(["like", "colors"])
    elif rand == 2:
        tags_per_users.append("colors")
    elif rand == 3:
        tags_per_users.append(["hashtag", "colors"])
    else:
        tags_per_users.append(["like, hashtag", "colors"])
    for filename in os.listdir(directory) :
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            ## First solution to get images a user liked
            rand = random.randint(1, 5)
            if rand == 1 :
                images_per_users_rand.append(filename)
            ## Second solution to get images u user liked
            if (json_data[filename]["orientation"] == "landscape"):
                images_per_users_orientation.append(filename)
            ## Third solution to get images u user liked
            if (json_data[filename]["colors"][0][0] >= 155):
                images_per_users_colors.append(filename)
    json_data_users[i] = {
        "images_rand": images_per_users_rand,
        "images_orientation": images_per_users_orientation,
        "images_colors": images_per_users_colors,
        "tags": tags_per_users
    }


In [41]:
# Creating dataframes to predict what the user might like
data = []
result_rand = []
result_orientation = []
result_colors = []
json_data_learning = dict(list(json_data.items())[:len(json_data)//3])

def get_result_dataframe(json_data, json_data_users, key):
    result = []
    for i in json_data:
        if i in json_data_users[0][key]:
            result.append('Favorite')
        else:
            result.append('NotFavorite')
    return result
        
result_rand = get_result_dataframe(json_data_learning, json_data_users, "images_rand")
result_orientation = get_result_dataframe(json_data_learning, json_data_users, "images_orientation")
result_colors = get_result_dataframe(json_data_learning, json_data_users, "images_colors")

for i in json_data_learning:        
    data.append([json_data_learning[i]["colors"][0][0],
              json_data_learning[i]["colors"][0][1],
              json_data_learning[i]["colors"][0][2],
              json_data_learning[i]["orientation"], 
              json_data_learning[i]["size"][0],
              json_data_learning[i]["size"][1]
             ])

dataframe = pd.DataFrame(data, columns=['Red', 'Green', 'Blue', 'Orientation', 'Width', 'Height'])
resultframe_rand = pd.DataFrame(result_rand, columns=['Liked'])
resultframe_orientation = pd.DataFrame(result_orientation, columns=['Liked'])
resultframe_colors = pd.DataFrame(result_colors, columns=['Liked'])

#generating numerical labels
le1 = LabelEncoder()
dataframe['Orientation'] = le1.fit_transform(dataframe['Orientation'])

le2 = LabelEncoder()
resultframe_rand['Liked'] = le2.fit_transform(resultframe_rand['Liked'])

le3 = LabelEncoder()
resultframe_orientation['Liked'] = le3.fit_transform(resultframe_orientation['Liked'])

le4 = LabelEncoder()
resultframe_colors['Liked'] = le4.fit_transform(resultframe_colors['Liked'])

# DTC
dtc_rand = tree.DecisionTreeClassifier()
dtc_rand = dtc_rand.fit(dataframe, resultframe_rand)

dtc_orientation = tree.DecisionTreeClassifier()
dtc_orientation = dtc_orientation.fit(dataframe, resultframe_orientation)

dtc_colors = tree.DecisionTreeClassifier()
dtc_colors = dtc_colors.fit(dataframe, resultframe_colors)

# RTC
rfc_rand = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0)
rfc_rand = rfc_rand.fit(dataframe, resultframe_rand.values.ravel())

rfc_orientation = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0)
rfc_orientation = rfc_orientation.fit(dataframe, resultframe_orientation.values.ravel())

rfc_colors = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0)
rfc_colors = rfc_colors.fit(dataframe, resultframe_colors.values.ravel())

dataframe_prediction = []
json_data_predicting = dict(list(json_data.items())[len(json_data)*2//3:])
for j in json_data_predicting:
    dataframe_prediction.append([json_data_predicting[j]["colors"][0][0],
              json_data_predicting[j]["colors"][0][1],
              json_data_predicting[j]["colors"][0][2],
              le1.transform([json_data_predicting[j]["orientation"]])[0], 
              json_data_predicting[j]["size"][0],
              json_data_predicting[j]["size"][1]
             ])
    
def get_prediction(method_object, label_encoder, dataframe_prediction):
    prediction = method_object.predict(dataframe_prediction) 
    return prediction


prediction_rand_dtc = get_prediction(dtc_rand, le1, dataframe_prediction)
prediction_orientation_dtc = get_prediction(dtc_orientation, le1, dataframe_prediction)
prediction_colors_dtc = get_prediction(dtc_colors, le1, dataframe_prediction)

prediction_rand_rfc = get_prediction(rfc_rand, le1, dataframe_prediction)
prediction_orientation_rfc = get_prediction(rfc_orientation, le1, dataframe_prediction)
prediction_colors_rfc = get_prediction(rfc_colors, le1, dataframe_prediction)
  

result_all_images = get_result_dataframe(json_data, json_data_users, "images_orientation")

accuracy_score(le3.transform(result_all_images[len(result_all_images)*2//3:]), prediction_orientation_dtc)

# une image => deux info : l'étiquette et un vecteur ac les infos de l'image cette tchoin
# étiquette 0 (aime pas) ou 1 (aime)
# find test split pr sklearn ça découpe les dataset automatiquement
# skleanr accuracy score pour ce que j'ai fait au desssus mais en mieux

NameError: name 'accuracy_score' is not defined